# MUTRAFF - Congestion Map Drawing
Congestion tracing based on Mutraff Experiments over Google Maps
References:
* Mutraff
* Jupyter gmaps: https://jupyter-gmaps.readthedocs.io/en/v0.3.3/gmaps.html
* Blog examples: https://people.revoledu.com/kardi/tutorial/Python/Displaying+Locations+using+Heatmap.html

## Imports section

In [127]:
import pandas as pd
import numpy as np
import googlemaps
import gmaps
import yaml

In [128]:
# https://martin-thoma.com/configuration-files-in-python/
def load_config( file ):
    config = {}
    with open(file, 'r') as ymlfile:
        config = yaml.load(ymlfile)
        ymlfile.close()
        # print(config)
    return config

In [129]:
cfg = load_config( "config.yaml")

# Import API_KEY from YAML
gm = googlemaps.Client(key=cfg['google_maps']['API_KEY'])
gmaps.configure(api_key=cfg['google_maps']['API_KEY']) # Your Google API key

## Definitions

In [130]:
MUTRAFF_HOME="/Users/alvaro/Desktop/workspace/mutraff/uah-gist-mutraff-bastra"
MUTRAFF_EXP_PATH=MUTRAFF_HOME + "/experiments/tmp"
SIMUL_TIME_STATS=None
COLUMNS=[]
exp={}
label={}

SMALL_FONT_SIZE=10
MID_FONT_SIZE=12
BIG_FONT_SIZE=14

## Functions Library

### Geocode and GeocodeStreetLocationCity functions
In case of receiving just edge names or address, the following functions will do the geocoding of street, location and the city.
The algorithm goes as follow:
* the code will try to search for street, location and city.
* If it is not succesful, it will try to search for street and the city.
* If it is still unsuccessful, it will search for location and city.
* If all of these attempts are not successful, it will produce fail in geocoding. 
* If any of the attempt is successful, it will append the result into the list and return this list of latitude and longitude.

In [131]:
def Geocode(query):
    # do geocoding
    try:
        geocode_result = gm.geocode(query)[0]       
        latitude = geocode_result['geometry']['location']['lat']
        longitude = geocode_result['geometry']['location']['lng']
        return latitude,longitude
    except IndexError:
        return 0
        
def GeocodeStreetLocationCity(data):
    lat=[]                            # initialize latitude list
    lng=[]                            # initialize longitude list
    start = data.index[0]             # start from the first data
    end = data.index[maxRow-1]        # end at maximum number of row
    #end = 100
    for i in range(start,end+1,1):    # iterate all rows in the data
        isSuccess=True                # initial Boolean flag
        
        #query = data.Street[i] + ',' + data.Location[i]
        query = data.Street[i] + ',' + 'Manila' # data.Location[i]
        # query = data.Location[i]
        # result=Geocode(query)
        result = 0
        if result==0:         # if not successful,
            print(i, 'is failed')
            isSuccess = False
        else:
            print(i, result)
        if isSuccess==True:           # if geocoding is successful,
            # store the results
            lat.append(result[0])     # latitude
            lng.append(result[1])     # longitude
    return lat,lng

### DrawHeatMap function
Creates a heat map to represent congestion data

In [132]:
def drawHeatMap( center_address, v_lat_long, v_weigths, zoom, intensity, radius):
    # do geocode for the whole mega city
    geocode_result = gm.geocode(center_address)[0]  # change the name into your city of interest

    # get the center of the city
    center_lat=geocode_result['geometry']['location']['lat']
    center_lng=geocode_result['geometry']['location']['lng']
    # print('center=',center_lat,center_lng)

    # setting the data and parameters
    # heatmap_layer = gmaps.heatmap_layer(locations, val, dissipating = True)
    heatmap_layer = gmaps.heatmap_layer(v_lat_long, v_weigths, dissipating = True)
    # heatmap_layer = gmaps.WeightedHeatmap(v_lat_long)
    heatmap_layer.disspating = True
    heatmap_layer.max_intensity = intensity
    heatmap_layer.point_radius = radius
    # draw the heatmap into a figure
    fig = gmaps.figure()
    fig = gmaps.figure(center = [center_lat,center_lng], zoom_level=zoom)
    fig.add_layer(heatmap_layer)
    return fig

### load_data function
Import csv data from experiments into valuable data

In [133]:
def load_data(experiment):
    global MUTRAFF_EXP_PATH
    
    theFile="edges.csv"
    filename = "{}/{}/{}".format(MUTRAFF_EXP_PATH,experiment,theFile)
    filename="alcalahenares.edges.csv"
    print("Parsing edges catalog "+filename)
    df1 = pd.read_csv(filename)
    
    theFile="edge_stats.csv"
    filename = "{}/{}/{}".format(MUTRAFF_EXP_PATH,experiment,theFile)
    print("Parsing data traffic file "+filename)
    df2 = pd.read_csv(filename)
    
    return df1, df2

## Experiments

In [134]:
exp[0]="alcalahenares_M_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit20_11_190508_003728"

exp[2200]="alcalahenares_M3h_nomaps_tele60_timeALL_fulltraffic_190510_235642"
exp[2201]="alcalahenares_M3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190511_163910"

exp[2300]="alcalahenares_L3h_nomaps_tele60_timeALL_fulltraffic_190515_130939"
exp[2301]="alcalahenares_L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_133434"

exp[2400]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_150939"
exp[2401]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_161514"

exp[2410]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_182031"
exp[2411]="alcalahenares_2L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190515_191357"
exp[2412]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_200544"

exp[2420]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_202233"
exp[2421]="alcalahenares_2L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190515_205714"
exp[2422]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_212938"

exp[2430]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_190516_084853"
exp[2431]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit10_01_190521_091537"
exp[2432]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit20_01_190521_143019"
exp[2433]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190516_091540"
exp[2434]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit100_01_190521_210550"

exp[2441]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit10_01_190521_213112"
exp[2442]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit20_01_190521_220448"
exp[2443]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190516_114311"
exp[2444]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit100_01_190521_223020"

exp[2500]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_190604_073604"
exp[2501]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_incident_190604_082709"
exp[2502]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit10_01_190604_091731"

In [135]:
label[2430]="Alcala 3L - No MuTraff maps"
label[2431]="Alcala 3L - 8  MuTraff maps - uniform(5) - 10% usage"
label[2432]="Alcala 3L - 8  MuTraff maps - uniform(5) - 20% usage"
label[2433]="Alcala 3L - 8  MuTraff maps - uniform(5) - 50% usage"
label[2434]="Alcala 3L - 8  MuTraff maps - uniform(5) - 100% usage"
label[2441]="Alcala 3L - 16 MuTraff maps - uniform(5) - 10% usage"
label[2442]="Alcala 3L - 16 MuTraff maps - uniform(5) - 20% usage"
label[2443]="Alcala 3L - 16 MuTraff maps - uniform(5) - 50% usage"
label[2444]="Alcala 3L - 16 MuTraff maps - uniform(5) - 100% usage"

label[2500]="Alcala 3L - no TWM"
label[2501]="Alcala 3L - no TWM - Incident"
label[2502]="Alcala 3L - no TWM 16 MuTraff maps - uniform(5) - 10% usage"

### Incident management

In [136]:
exp[2500]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_incident_190603_000953"
label[2500]="Alcala 3L - No MuTraff maps - Incident 30' main roundabout"

### FORESIGHT AND TWM

In [137]:
exp[2599]="alcalahenares_3L3h_nomaps_timeALL_taz5-taz50_01_190605_225531"
exp[2600]="alcalahenares_3L3h_nomaps_timeALL_taz5-taz50_01_190607_163410"
exp[2601]="alcalahenares_3L3h_nomaps_timeALL_taz5-taz50_incident_01_190605_233312"
exp[2602]="alcalahenares_3L3h_nomaps_timeALL_taz5-taz50_foresight_01_190606_072936"
exp[2603]="alcalahenares_3L3h_nomaps_timeALL_taz5-taz50_foresight_incident_01_190606_082447"

label[2599]="No TWM - Alcala - Taz5 to Taz50 - No foresight - Exp1"
label[2600]="No TWM - Alcala - Taz5 to Taz50 - No foresight - Exp2"
label[2601]="No TWM - Alcala - Taz5 to Taz50 - No foresight - INCIDENT"
label[2602]="No TWM - Alcala - Taz5 to Taz50 - Fore2-3-4-10"
label[2603]="No TWM - Alcala - Taz5 to Taz50 - Fore2-3-4-10 - INCIDENT"

In [138]:
experiment=exp[2430]
experiment=exp[2500]
experiment=exp[2601]

In [139]:
exp[1170]="alcalahenares_XL_nomutraff_tele60_nomaps_fulltraffic_170206_085516"
label[1170]="alcalahenares(XL)  No TWM -  -  "

## Pollution Measures

In [140]:
exp[3000]="grid16EMISSIONS_noBastra_reference_fulltraffic_01_190817_222335"
label[3000]="GRID16 - FullTraffic - NoTWM"

In [141]:
exp[1183]="alcalahenares_3L3h_TopEdges_timeALL_fulltraffic_01_190929_231834"
label[1183]="xxx"

In [142]:
experiment=exp[1183]

## Emergencies Management

In [143]:
exp[3001]="alcalahenares_Emergency_noTWM_L_T60_01_191123_103505"
label[3001]="noTWM/T60"

In [9]:
exp[3002]="alcalahenares_Emergency_TWM50_L_T60_01_191123_103510"
label[3002]="TWM50/T60"

In [100]:
exp[3003]="alcalahenares_Emergency_TWM100_L_T60_penalties_01_191123_185635"
label[3001]="noTWM/T60"

In [144]:
experiment=exp[3001]

## Part 1 : Load traffic data from measures or experiments

In [145]:
edges, stats = load_data( experiment )

Parsing edges catalog alcalahenares.edges.csv
Parsing data traffic file /Users/alvaro/Desktop/workspace/mutraff/uah-gist-mutraff-bastra/experiments/tmp/alcalahenares_Emergency_noTWM_L_T60_01_191123_103505/edge_stats.csv


In [146]:
edges.head()

,edge_id,odmat_id,from_node,to_node,from_lat,from_long,to_lat,to_long,center_lat,center_long,name
0,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,NaN
1,2,-100851688,1165482328,1165482290,40.476372,-3.393535,40.476832,-3.391995,40.476602,-3.392765,NaN
2,3,-100851690,1165482325,1165482312,40.475548,-3.396469,40.476015,-3.394803,40.475782,-3.395636,NaN
3,4,-100851695,1165482322,1165482272,40.468375,-3.405725,40.469638,-3.401857,40.469007,-3.403791,NaN
4,5,-103584157,311472409,1195985289,40.484626,-3.396309,40.485168,-3.396721,40.484897,-3.396515,NaN


* Latitud: 40.4994044

* Longitud: -3.3401732

In [166]:
print("Max time={}".format(stats['time'].max()))
print(stats.columns)
stats.head()


Max time=2952
Index(['time', 'edge_id', 'traf_travel_time', 'traf_waiting_time',
       'traf_total_veh_num', 'traf_halted_veh_num', 'traf_av_occupancy',
       'traf_av_speed', 'emission_co2', 'emission_co', 'emission_hc',
       'emission_noise', 'emission_nox', 'emission_PMx', 'consum_epower',
       'consum_fuel'],
      dtype='object')


,time,edge_id,traf_travel_time,traf_waiting_time,traf_total_veh_num,traf_halted_veh_num,traf_av_occupancy,traf_av_speed,emission_co2,emission_co,emission_hc,emission_noise,emission_nox,emission_PMx,consum_epower,consum_fuel
0,0,0,4.122030,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,5.040677,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,5.411807,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,25.655868,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,4.825054,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### drawTrafficInstantShot
Prints a heatmap with an spatial and time exposure

In [148]:
def drawInstantShot( city, dataset, timestamp, variable_name ):
    time_shot = dataset[dataset['time']==timestamp].groupby(['time','edge_id']).sum() # .unstack()['traf_halted_veh_num']
    v_lat_long = time_shot[['center_lat','center_long']]
    v_weight = time_shot[variable_name]
    
    print( "Lat/Long size={}".format( len(v_lat_long) ))
    print( "Weight size={}".format( len(v_weight) ))
    # set up parameters
    zoom=13
    max_intensity=float(np.max(v_weight))
    radius=10

    # call the function to draw the heatmap
    drawHeatMap(city, v_lat_long, v_weight, zoom, max_intensity, radius)

merge informacion together

In [149]:
dataset = pd.merge(edges, stats, left_on='edge_id', right_on='edge_id')

In [150]:
#time=2520
#drawInstantShot( 'Alcala de Henares', dataset, time, 'traf_halted_veh_num' )

### Join data and extract a time snapshot
Summarize all the data corresponding to the *timestamp* time and combines this data with edge values.

In [168]:
timestamp=3000
timestamp=2520
timestamp=2520

stats[stats['time']<=timestamp].head()

,time,edge_id,traf_travel_time,traf_waiting_time,traf_total_veh_num,traf_halted_veh_num,traf_av_occupancy,traf_av_speed,emission_co2,emission_co,emission_hc,emission_noise,emission_nox,emission_PMx,consum_epower,consum_fuel
0,0,0,4.122030,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,5.040677,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,5.411807,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,25.655868,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,4.825054,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
time_stats_shot  = stats[stats['time']<=timestamp].groupby(['time','edge_id']).sum() # .unstack()['traf_halted_veh_num']
#time_stats_shot
dataset = pd.merge(edges, time_stats_shot, left_on='edge_id', right_on='edge_id')

In [170]:
print(dataset.columns)
dataset.head()

Index(['edge_id', 'odmat_id', 'from_node', 'to_node', 'from_lat', 'from_long',
       'to_lat', 'to_long', 'center_lat', 'center_long', 'name',
       'traf_travel_time', 'traf_waiting_time', 'traf_total_veh_num',
       'traf_halted_veh_num', 'traf_av_occupancy', 'traf_av_speed',
       'emission_co2', 'emission_co', 'emission_hc', 'emission_noise',
       'emission_nox', 'emission_PMx', 'consum_epower', 'consum_fuel'],
      dtype='object')


,edge_id,odmat_id,from_node,to_node,from_lat,from_long,to_lat,to_long,center_lat,center_long,...,traf_av_occupancy,traf_av_speed,emission_co2,emission_co,emission_hc,emission_noise,emission_nox,emission_PMx,consum_epower,consum_fuel
0,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,...,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,...,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,...,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,...,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,...,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
# stats['time']

## Part 3 : Draw the heatmap

In [171]:
OBJECTIVE='traf_halted_veh_num'
OBJECTIVE='traf_total_veh_num'
# df=pd.read_csv('locations.csv', low_memory=False, index_col = 'key')  # read geocoded location from file
v_lat_long = dataset[['center_lat','center_long']]
v_weight = dataset[OBJECTIVE]


### Drawing the map
To call the function drawHeatMap, set up the parameters and call the function to draw the heatmap. You can experiment with different parameter values. The result map can also be saved as PNG file.

In [173]:
# set up parameters
zoom=13
max_intensity=float(np.max(v_weight))
radius=10

# call the function to draw the heatmap
drawHeatMap('Alcala de Henares', v_lat_long, v_weight, zoom, max_intensity, radius)

Figure(layout=FigureLayout(height='420px'))

# Retocado del mapa para los experimentos

In [115]:
filename="{}_out.csv".format(experiment)
dataset.to_csv(filename, sep=';')
print("dumped file {}".format(filename))

dumped file alcalahenares_Emergency_TWM100_L_T60_penalties_01_191123_185635_out.csv


In [116]:
filename="{}_out.xls".format(experiment)
dataset.to_excel(filename)
print("dumped file {}".format(filename))

dumped file alcalahenares_Emergency_TWM100_L_T60_penalties_01_191123_185635_out.xls


In [123]:
data2 = pd.read_excel (filename, sheet_name='Sheet1')
print(data2.columns)
data2.head()

Index(['ID', 'edge_id', 'odmat_id', 'from_node', 'to_node', 'from_lat',
       'from_long', 'to_lat', 'to_long', 'center_lat', 'center_long', 'name',
       'traf_travel_time', 'traf_waiting_time', 'traf_total_veh_num',
       'traf_halted_veh_num', 'traf_av_occupancy', 'traf_av_speed',
       'emission_co2', 'emission_co', 'emission_hc', 'emission_noise',
       'emission_nox', 'emission_PMx', 'consum_epower', 'consum_fuel', 'TAZ',
       'OVERLOAD', 'OBJECTIVE'],
      dtype='object')


,ID,edge_id,odmat_id,from_node,to_node,from_lat,from_long,to_lat,to_long,center_lat,...,emission_co,emission_hc,emission_noise,emission_nox,emission_PMx,consum_epower,consum_fuel,TAZ,OVERLOAD,OBJECTIVE
0,0,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,...,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0
1,1,2,-100851688,1165482328,1165482290,40.476372,-3.393535,40.476832,-3.391995,40.476602,...,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0
2,2,3,-100851690,1165482325,1165482312,40.475548,-3.396469,40.476015,-3.394803,40.475782,...,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0
3,3,4,-100851695,1165482322,1165482272,40.468375,-3.405725,40.469638,-3.401857,40.469007,...,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0
4,4,5,-103584157,311472409,1195985289,40.484626,-3.396309,40.485168,-3.396721,40.484897,...,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0


In [124]:
v_lat_long = data2[['center_lat','center_long']]
v_weight = data2['OBJECTIVE']
# set up parameters
zoom=13
max_intensity=float(np.max(v_weight))
radius=10

# call the function to draw the heatmap
drawHeatMap('Alcala de Henares', v_lat_long, v_weight, zoom, max_intensity, radius)

Figure(layout=FigureLayout(height='420px'))